<a href="https://colab.research.google.com/github/jokefun022/Multi-Model-Cyberbullying-Detection-in-Roman-Urdu-using-Deep-Learning/blob/main/My_original_project_19_10_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install numpy pandas scikit-learn matplotlib seaborn
# !pip install torch torchvision torchaudio
# !pip install tensorflow tensorflow-text tensorflow-decision-forests tf-keras
# !pip install gensim transformers huggingface_hub
# !pip install emoji joblib

In [2]:
# # Colab cell 1: install and download necessary data
# !pip uninstall -y numpy pandas scikit-learn gensim transformers torch tensorflow emoji matplotlib seaborn huggingface_hub joblib tensorflow-text tensorflow-decision-forests tf-keras scipy
# !pip install -q numpy==1.26.4 pandas==2.2.2 scikit-learn==1.4.2 transformers==4.41.2 torch==2.2.1 emoji==2.8.0 matplotlib==3.8.4 seaborn==0.13.1 huggingface_hub==0.23.0 joblib==1.4.2 scipy==1.7.3
# !pip install -q gensim==4.3.2

# !python -m nltk.downloader punkt stopwords

# import nltk
# nltk.download('stopwords', quiet=True)
# nltk.download('punkt', quiet=True)

In [3]:
!pip install numpy pandas scikit-learn matplotlib seaborn

In [4]:
!pip install torch torchvision torchaudio

In [5]:
!pip install tensorflow tensorflow-text tensorflow-decision-forests tf-keras

In [6]:
!pip install gensim transformers huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 47.7 MB/s eta 0:00:00


In [7]:
!pip install emoji joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 16.7 MB/s eta 0:00:00


In [8]:
# System & utilities
import os, gc, time, math, re, string, warnings
import pandas as pd
import numpy as np

# Sklearn - preprocessing, models, metrics
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             classification_report, confusion_matrix, roc_auc_score)
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight

# ML models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB

# DL embeddings & NLP
from gensim.models import Word2Vec, FastText
from gensim.models import KeyedVectors  # for pre-trained embeddings
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
import joblib
from tqdm import tqdm

# Transformers (Hugging Face)
import torch
from transformers import (AutoTokenizer, AutoModel, AutoModelForSequenceClassification,
                          Trainer, TrainingArguments)

# Hugging Face Hub
from huggingface_hub import HfApi, HfFolder, Repository, login

# Keras / TensorFlow (DL models)
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Embedding, Dense, Dropout, Conv1D, MaxPooling1D, Flatten,
                                     LSTM, GRU, SimpleRNN, Bidirectional, GlobalAveragePooling1D)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

print("✅ All libraries imported successfully")
print("Torch:", torch.__version__, " | TensorFlow:", tf.__version__)


✅ All libraries imported successfully
Torch: 2.8.0+cu126  | TensorFlow: 2.19.0


In [9]:
# Colab cell 2: imports
import os, gc, time, math
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight

# ML models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

# DL & embeddings
# from gensim.models import Word2Vec, FastText # Commenting out gensim imports
import emoji
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Transformers
# import torch # Commenting out torch and transformers for now
# from transformers import AutoTokenizer, AutoModel

# Hugging Face
# from huggingface_hub import HfApi, HfFolder, Repository, login

# Keras (for DL models)
# import tensorflow as tf # Commenting out tensorflow and keras for now
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM, GRU, SimpleRNN, Bidirectional
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

print("Attempting minimal imports...")

Attempting minimal imports...


In [10]:
DATA_PATH = "/content/Complete_Data_With_Emoji (1).csv"  # path in Colab
df = pd.read_csv(DATA_PATH, encoding="utf-8")
# adjust column names here if necessary:
TEXT_COL = "Tweet_Text_With_Emoji"
LABEL_COL = "Label"

print("Rows:", len(df))
df.head()

Rows: 12672


,Tweet_Text_With_Emoji,Label,Sentiment Analysis
0,kah kay yih sach 😊 hai,0,Neutral
1,neither bhadwaa 😑 tantra rather madarjaat log ...,0,Neutral
2,kyoon okay yih kuchh logo ki 😃 fitrat hai,0,Neutral
3,lagta hsi man 🙌 chaaha job naey day rahay bhaa...,0,Neutral
4,achchha na bataao mujhay pata hai 😊,0,Neutral


In [11]:
# Label encode
le = LabelEncoder()
df['label_enc'] = le.fit_transform(df[LABEL_COL].astype(str))
num_classes = len(le.classes_)
print("Classes:", le.classes_, "Num:", num_classes)

Classes: ['0' '1' '2' '3' '4' '5'] Num: 6


In [12]:
RND = 42
X = df[TEXT_COL].astype(str).tolist()
y = df['label_enc'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RND, stratify=y)
print("Train:", len(X_train), "Test:", len(X_test))


Train: 10137 Test: 2535


In [13]:
# 3A: TF-IDF
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)
print("TF-IDF shapes:", X_train_tfidf.shape, X_test_tfidf.shape)

# 3B: Word2Vec (train on full texts tokenized)
sentences = [s.split() for s in X]  # simple whitespace tokenization; adapt if you have better tokenizer
w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4, epochs=10)
wv_dim = w2v.wv.vector_size

def avg_w2v_features(texts, model=w2v, dim=wv_dim):
    fv = []
    for t in texts:
        words = t.split()
        vecs = [model.wv[w] for w in words if w in model.wv]
        if vecs:
            fv.append(np.mean(vecs, axis=0))
        else:
            fv.append(np.zeros(dim))
    return np.vstack(fv)

X_train_w2v = avg_w2v_features(X_train)
X_test_w2v  = avg_w2v_features(X_test)
print("W2V shapes:", X_train_w2v.shape, X_test_w2v.shape)

# 3C: FastText
ft = FastText(sentences, vector_size=100, window=5, min_count=2, workers=4, epochs=10)
ft_dim = ft.wv.vector_size

def avg_ft_features(texts, model=ft, dim=ft_dim):
    fv = []
    for t in texts:
        words = t.split()
        vecs = [model.wv[w] for w in words if w in model.wv]
        if vecs:
            fv.append(np.mean(vecs, axis=0))
        else:
            fv.append(np.zeros(dim))
    return np.vstack(fv)

X_train_ft = avg_ft_features(X_train)
X_test_ft = avg_ft_features(X_test)
print("FastText shapes:", X_train_ft.shape, X_test_ft.shape)

# 3D: Emoji2Vec - simple approach: extract only emojis and run char ngram TF-IDF on emoji strings
import re
EMOJI_PATTERN = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
                           "]+", flags=re.UNICODE)

def extract_emojis(text):
    ems = EMOJI_PATTERN.findall(text)
    return " ".join(ems) if ems else ""

X_train_emoji = [extract_emojis(t) for t in X_train]
X_test_emoji  = [extract_emojis(t) for t in X_test]

# If no emojis in some rows, they will be empty strings; use char-level TF-IDF
emoji_vec = TfidfVectorizer(analyzer='char', ngram_range=(1,3), min_df=1)
X_train_emoji_tf = emoji_vec.fit_transform(X_train_emoji)
X_test_emoji_tf  = emoji_vec.transform(X_test_emoji)
print("Emoji TF shapes:", X_train_emoji_tf.shape, X_test_emoji_tf.shape)

# 3E: BERT (XLM-R) CLS embeddings (batched)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
bert = AutoModel.from_pretrained("xlm-roberta-base").to(device)
bert.eval()

def get_bert_cls(texts, batch_size=16):
    emb = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(batch, truncation=True, padding=True, max_length=128, return_tensors="pt").to(device)
        with torch.no_grad():
            out = bert(**enc)
        cls = out.last_hidden_state[:,0,:].cpu().numpy()
        emb.append(cls)
    return np.vstack(emb)

# Be careful: this can take time and memory for large datasets. We'll compute full if feasible.
print("Computing BERT embeddings for train & test (this may take some time)...")
X_train_bert = get_bert_cls(X_train, batch_size=16)
X_test_bert  = get_bert_cls(X_test, batch_size=16)
print("BERT shapes:", X_train_bert.shape, X_test_bert.shape)


TF-IDF shapes: (10137, 5000) (2535, 5000)
W2V shapes: (10137, 100) (2535, 100)
FastText shapes: (10137, 100) (2535, 100)
Emoji TF shapes: (10137, 114) (2535, 114)
Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Computing BERT embeddings for train & test (this may take some time)...
BERT shapes: (10137, 768) (2535, 768)


In [14]:
import itertools
def eval_and_report(name, y_true, y_pred, labels_list=None, output_prefix=None):
    acc = accuracy_score(y_true, y_pred)
    prec_macro = precision_score(y_true, y_pred, average='macro', zero_division=0)
    rec_macro = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0)
    print(f"\n{name} | Acc: {acc:.4f} | Prec(macro): {prec_macro:.4f} | Rec(macro): {rec_macro:.4f} | F1(macro): {f1_macro:.4f}")
    print(classification_report(y_true, y_pred, target_names=labels_list if labels_list is not None else le.classes_))
    # confusion matrix
    # cm = confusion_matrix(y_true, y_pred)
    # plt.figure(figsize=(6,5))
    # sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=(labels_list if labels_list else le.classes_), yticklabels=(labels_list if labels_list else le.classes_))
    # plt.xlabel("Predicted"); plt.ylabel("True"); plt.title(f"Confusion Matrix: {name}")
    # if output_prefix:
    #     plt.savefig(f"{output_prefix}_confusion.png", bbox_inches='tight')
    # plt.show()
    return {"accuracy": acc, "precision_macro": prec_macro, "recall_macro": rec_macro, "f1_macro": f1_macro}

In [26]:
# Prepare feature matrix dictionary for convenience
features = {
    # "TFIDF": (X_train_tfidf, X_test_tfidf),
    "Word2Vec": (X_train_w2v, X_test_w2v),
    # "FastText": (X_train_ft, X_test_ft),
    "Emoji2Vec": (X_train_emoji_tf, X_test_emoji_tf),
    "BERT": (X_train_bert, X_test_bert)
}

ml_models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, class_weight='balanced'),
    "SVM": LinearSVC(class_weight='balanced', max_iter=2000),
    "RandomForest": RandomForestClassifier(n_estimators=200, n_jobs=-1, class_weight='balanced'),
    "DecisionTree": DecisionTreeClassifier(class_weight='balanced'),
    # "GradientBoosting": GradientBoostingClassifier(),
    # "NaiveBayes": MultinomialNB()
}

results = []
os.makedirs("conf_matrices", exist_ok=True)

for feat_name, (Xtr, Xte) in features.items():
    print("\n\n==== FEATURE:", feat_name, "====")
    # If sparse (TF-IDF, emoji TF), OK; if dense numpy, it's fine too.
    for model_name, model in ml_models.items():
        print("\n>> Model:", model_name)
        # Some models (MultinomialNB) need non-negative inputs; ensure TFIDF or emoji features used for NB
        try:
            model_instance = model
            model_instance.fit(Xtr, y_train)
            y_pred = model_instance.predict(Xte)
            r = eval_and_report(f"{model_name} + {feat_name}", y_test, y_pred, output_prefix=f"conf_matrices/{model_name}_{feat_name}")
            r.update({"model": model_name, "feature": feat_name})
            results.append(r)
            # Save model file
            joblib.dump(model_instance, f"models/{model_name}_{feat_name}.joblib")
        except Exception as e:
            print("Error training", model_name, feat_name, "->", e)
        gc.collect()

# Save results DataFrame
os.makedirs("models", exist_ok=True)
results_df = pd.DataFrame(results)
results_df.to_csv("ml_results_summary.csv", index=False)
results_df.sort_values("accuracy", ascending=False).head(20)




==== FEATURE: Word2Vec ====

>> Model: LogisticRegression

LogisticRegression + Word2Vec | Acc: 0.8201 | Prec(macro): 0.6552 | Rec(macro): 0.7499 | F1(macro): 0.6836
              precision    recall  f1-score   support

           0       0.96      0.90      0.93      1211
           1       0.54      0.72      0.62       145
           2       0.78      0.67      0.72       628
           3       0.39      0.61      0.47        80
           4       0.31      0.68      0.42        71
           5       0.96      0.92      0.93       400

    accuracy                           0.82      2535
   macro avg       0.66      0.75      0.68      2535
weighted avg       0.85      0.82      0.83      2535


>> Model: SVM

SVM + Word2Vec | Acc: 0.8951 | Prec(macro): 0.7749 | Rec(macro): 0.8061 | F1(macro): 0.7878
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1211
           1       0.66      0.74      0.70       145
           2      

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



LogisticRegression + BERT | Acc: 0.7822 | Prec(macro): 0.6308 | Rec(macro): 0.7645 | F1(macro): 0.6701
              precision    recall  f1-score   support

           0       0.93      0.80      0.86      1211
           1       0.44      0.72      0.54       145
           2       0.85      0.69      0.76       628
           3       0.40      0.80      0.53        80
           4       0.35      0.68      0.46        71
           5       0.82      0.90      0.86       400

    accuracy                           0.78      2535
   macro avg       0.63      0.76      0.67      2535
weighted avg       0.83      0.78      0.80      2535


>> Model: SVM

SVM + BERT | Acc: 0.8777 | Prec(macro): 0.7744 | Rec(macro): 0.8107 | F1(macro): 0.7903
              precision    recall  f1-score   support

           0       0.92      0.93      0.92      1211
           1       0.64      0.66      0.65       145
           2       0.90      0.82      0.86       628
           3       0.67      0.8

,accuracy,precision_macro,recall_macro,f1_macro,model,feature
1,0.895069,0.774907,0.806069,0.787839,SVM,Word2Vec
9,0.877712,0.774368,0.810744,0.790259,SVM,BERT
2,0.850493,0.860667,0.700784,0.757678,RandomForest,Word2Vec
0,0.820118,0.655162,0.749949,0.683554,LogisticRegression,Word2Vec
8,0.782249,0.630778,0.764490,0.670058,LogisticRegression,BERT
5,0.747929,0.684455,0.669401,0.670349,SVM,Emoji2Vec
4,0.746746,0.682616,0.674165,0.671444,LogisticRegression,Emoji2Vec
3,0.744379,0.619375,0.634688,0.625704,DecisionTree,Word2Vec
10,0.724260,0.866091,0.461467,0.533281,RandomForest,BERT
6,0.677712,0.665252,0.683822,0.593874,RandomForest,Emoji2Vec


In [27]:
# Tokenize everything
MAX_WORDS = 20000
MAX_LEN = 80
tk = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tk.fit_on_texts(X)  # fit on whole corpus
vocab_size = min(MAX_WORDS, len(tk.word_index)+1)
print("Vocab size:", vocab_size)

X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')

# Prepare embedding matrices for Word2Vec / FastText
def build_embedding_matrix(model, token_index, dim):
    emb_matrix = np.zeros((vocab_size, dim))
    for word, i in token_index.items():
        if i>=vocab_size: continue
        if word in model.wv:
            emb_matrix[i] = model.wv[word]
        else:
            emb_matrix[i] = np.random.normal(scale=0.6, size=(dim,))
    return emb_matrix

w2v_emb_matrix = build_embedding_matrix(w2v, tk.word_index, wv_dim)
ft_emb_matrix  = build_embedding_matrix(ft, tk.word_index, ft_dim)

y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat  = to_categorical(y_test, num_classes=num_classes)


Vocab size: 19699


In [28]:
# ----------------------------------------------
# ✅ STEP 1: Create or Load the Embedding Matrices
# ----------------------------------------------

# If Word2Vec already defined
# w2v_emb_matrix = ...

# --- Emoji2Vec Embedding Matrix ---
try:
    emoji2vec_matrix
except NameError:
    print("⚠️ emoji2vec_matrix not found. Initializing random matrix for now.")
    emoji2vec_matrix = np.random.uniform(-0.25, 0.25, (vocab_size, 300))

# --- BERT Embedding Matrix ---
try:
    bert_emb_matrix
except NameError:
    print("⚠️ bert_emb_matrix not found. Initializing random matrix for now.")
    bert_emb_matrix = np.random.uniform(-0.25, 0.25, (vocab_size, 768))

# ----------------------------------------------
# ✅ STEP 2: Define Your DL Model Function
# ----------------------------------------------
def make_dl_model(model_type, embedding_matrix=None, trainable_emb=False, input_len=MAX_LEN, emb_dim=100, num_classes=num_classes):
    model = Sequential()

    # Embedding Layer
    if embedding_matrix is not None:
        model.add(Embedding(input_dim=vocab_size,
                            output_dim=embedding_matrix.shape[1],
                            input_length=input_len,
                            weights=[embedding_matrix],
                            trainable=trainable_emb))
    else:
        model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_len))

    # Model selection
    if model_type == "CNN":
        model.add(Conv1D(128, 5, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
    elif model_type == "RNN":
        model.add(SimpleRNN(128))
    elif model_type == "LSTM":
        model.add(LSTM(128))
    elif model_type == "BiLSTM":
        model.add(Bidirectional(LSTM(128)))
    elif model_type == "GRU":
        model.add(GRU(128))

    # Dense Layers
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


# ----------------------------------------------
# ✅ STEP 3: Define DL Models & Embedding Configs
# ----------------------------------------------
dl_models = ["CNN", "RNN", "LSTM", "BiLSTM", "GRU"]

emb_configs = {
    "Word2Vec": w2v_emb_matrix,           # Word2Vec embeddings
    "Emoji2Vec": emoji2vec_matrix,   # Emoji embeddings
    "BERT": bert_emb_matrix          # BERT contextual embeddings
}

dl_results = []

# ----------------------------------------------
# ✅ STEP 4: Model Training Loop
# ----------------------------------------------
for emb_name, emb_matrix in emb_configs.items():
    for mtype in dl_models:
        print(f"\n=== DL: {mtype} with {emb_name} embeddings ===")

        model = make_dl_model(mtype, embedding_matrix=emb_matrix, trainable_emb=False, emb_dim=100)

        es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
        ckpt_path = f"models/dl_{mtype}_{emb_name}.h5"
        mc = ModelCheckpoint(ckpt_path, monitor='val_loss', save_best_only=True)

        history = model.fit(
            X_train_pad, y_train_cat,
            validation_split=0.1,
            epochs=8, batch_size=64,
            callbacks=[es, mc], verbose=2
        )

        preds_proba = model.predict(X_test_pad)
        preds = preds_proba.argmax(axis=1)

        metrics = eval_and_report(f"{mtype} + {emb_name}", y_test, preds,
                                  output_prefix=f"conf_matrices/{mtype}_{emb_name}")
        metrics.update({"model": mtype, "feature": emb_name})
        dl_results.append(metrics)

        model.save(ckpt_path)
        gc.collect()


# Save results DataFrame
os.makedirs("models", exist_ok=True)
dl_results_df = pd.DataFrame(dl_results)
dl_results_df.to_csv("dl_results_summary.csv", index=False)
dl_results_df.sort_values("accuracy", ascending=False).head(20)


=== DL: CNN with Word2Vec embeddings ===
Epoch 1/8


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


143/143 - 6s - 43ms/step - accuracy: 0.6892 - loss: 0.8903 - val_accuracy: 0.8442 - val_loss: 0.5092
Epoch 2/8


143/143 - 1s - 6ms/step - accuracy: 0.8506 - loss: 0.4793 - val_accuracy: 0.8777 - val_loss: 0.4028
Epoch 3/8


143/143 - 1s - 4ms/step - accuracy: 0.8714 - loss: 0.3955 - val_accuracy: 0.8866 - val_loss: 0.3669
Epoch 4/8


...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_16['feature'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_16, x='index', y='feature', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_17['model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_17, x='accuracy', y='model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['feature'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='accuracy', y='feature', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [33]:
# Function to plot training history
def plot_history(history, model_name, emb_name):
    plt.figure(figsize=(12, 5))

    # Plot Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{model_name} with {emb_name}: Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Plot Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{model_name} with {emb_name}: Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# To use this function, you would call it after training each model in your loop:
# plot_history(history, mtype, emb_name)

# Example of how to call it with the last trained model's history (replace with your loop variable if needed)
# Assuming 'history' variable still holds the history of the last trained model from cell lG1fLbKHkM1h
# You would need to store histories in a dictionary if you want to plot all of them after the loop.
# For demonstration, let's assume history from the last model run is available.
# Note: To plot for all models, you would need to collect history objects during the training loop
# and iterate through them here.

# Example call (assuming history, mtype, emb_name are available from the last iteration):
# print("\nPlotting history for the last trained model:")
# plot_history(history, mtype, emb_name)

# A better approach would be to modify the training loop to store histories:
# dl_histories = {}
# for emb_name, emb_matrix in emb_configs.items():
#     for mtype in dl_models:
#         ... training code ...
#         history = model.fit(...)
#         dl_histories[f"{mtype}_{emb_name}"] = history
#         ... evaluation and saving ...

# Then, iterate through dl_histories to plot:
# for name, hist in dl_histories.items():
#     model_name, emb_name = name.split('_', 1) # Assuming format like "CNN_Word2Vec"
#     plot_history(hist, model_name, emb_name)